In [ ]:
import pandas as pd
import json
import numpy as np

# ⚠️ Please see ReadMe.md before attempting to modify code here! ⚠️

# If adding another year, append to this array here
years = ['2018', '2019', '2020', '2021', '2022', '2023']
masterDict = dict()
for curYear in years:
    df = pd.read_excel('student_wages_tags.xlsx',sheet_name=curYear)
    print("✅ Excel read success for year ", curYear)   
    masterList = []
    for index, row in df.iterrows():  
        workgp = row['Workgroup']
        if workgp == np.nan or pd.isnull(workgp) or "EMPTY" in str(workgp).upper():          
            workgp = ""

        edu = row['Category Status']
        if edu == "Undergraduate Student":
            edu = "undergraduate"
        else:
            edu = "graduate"

        unitSplit = str(row['Unit']).split("-")
        
        tagList = []
        tag = row['tags']

        if tag == np.nan or pd.isnull(tag) or "EMPTY" in str(tag).upper():    
            tag = "" 
            tagList = []
        else:
            tag = str(tag)
            for t in tag.split(","):
                tagList.append(t.strip())
        
        obj = {
            'education': edu,
            'wage': row['Hour Rate'],
            'unit': unitSplit[1],
            'workgroup': workgp,
            'department': unitSplit[0],
            'year': curYear,
            'tags': tagList
            }

        masterList.append(obj)
    
    masterDict[curYear] = masterList

# Serializing json
json_object = json.dumps(masterDict, indent=4)
 
# Writing to sample.json
with open("../exports/sample_year.json", "w") as outfile:
    outfile.write(json_object)

print("Write success ✅")

In [ ]:
import json
f = open('../exports/sample_year.json')
data = json.load(f)

# Data structures

set_department = set()
set_unit = set()
set_workgroup = set()

for year in ['2018', '2019', '2020', '2021', '2022']:
    
    for entry in data[year]:
        set_department.add(entry['department'].strip())
        set_unit.add(entry['unit'].strip())
        set_workgroup.add(entry['workgroup'].strip())


# set_department.remove("")
# set_unit.remove("")
set_workgroup.remove("")

export = dict()
export['unique_department'] = list(set_department)
export['unique_unit'] = list(set_unit)
export['unique_workgroup'] = list(set_workgroup)

export['unique_department'].sort()
export['unique_unit'].sort()
export['unique_workgroup'].sort()

# Serializing json
json_object = json.dumps(export, indent=4)
 
# Writing to sample.json
with open("../exports/sample_metadata.json", "w") as outfile:
    outfile.write(json_object)

print("Write success ✅")

In [ ]:

# Data structures

unitToWorkgroup = dict()

for unit in export['unique_unit']:
    unitToWorkgroup[unit] = set()

for entry in data['2022']:   
    unitToWorkgroup[entry['unit']].add(entry['workgroup'])

# print(unitToWorkgroup)



for unit in unitToWorkgroup.keys():
    unitToWorkgroup[unit] = list(unitToWorkgroup[unit])
    unitToWorkgroup[unit].sort()

json_object = json.dumps(unitToWorkgroup, indent=4)

# Writing to sample.json
with open("../exports/unitToWorkgroup.json", "w") as outfile:
    outfile.write(json_object)

print("Write success ✅")

In [ ]:
departmentToUnit = dict()

for dept in export['unique_department']:
    departmentToUnit[dept] = set()

for entry in data['2022']:   
    departmentToUnit[entry['department']].add(entry['unit'])

# print(unitToWorkgroup)


for dept in departmentToUnit.keys():
    departmentToUnit[dept] = list(departmentToUnit[dept])
    departmentToUnit[dept].sort()

json_object = json.dumps(departmentToUnit, indent=4)

# Writing to sample.json
with open("../exports/departmentToUnit.json", "w") as outfile:
    outfile.write(json_object)

print("Write success ✅")

In [ ]:
from bs4 import BeautifulSoup
import requests
import re
  
  
# function to extract html document from given url
def getHTMLdocument(url):
      
    # request for HTML document of given url
    response = requests.get(url)
      
    # response will be provided in JSON format
    return response.text
  
    
# assign required credentials
# assign URL
url_to_scrape = "https://ejobs.umd.edu/assets/reset-fonts-0913b2708033472b5c49ea0865a9e34ec5efe0ecf8aaaeda2097e79ae4e9efc9.css"
  
# create document
html_document = getHTMLdocument(url_to_scrape)
print(html_document)